In [1]:
%pip install git+https://github.com/tinygrad/tinygrad.git


  Cloning https://github.com/tinygrad/tinygrad.git to /private/var/folders/x4/ygyvps6n5rdf76sm9_9t__gw0000gn/T/pip-req-build-4lkf1e94
  Running command git clone --filter=blob:none --quiet https://github.com/tinygrad/tinygrad.git /private/var/folders/x4/ygyvps6n5rdf76sm9_9t__gw0000gn/T/pip-req-build-4lkf1e94
  Resolved https://github.com/tinygrad/tinygrad.git to commit 07ec99001a97098bdb0cd5ab484137eddf3d16b5
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tinygrad import Device
Device.DEFAULT = "CLANG" 
print(Device.DEFAULT)

CLANG


In [3]:
from tinygrad import Tensor, nn

class Model:
  def __init__(self):
    self.l1 = nn.Conv2d(1, 32, kernel_size=(3,3))
    self.l2 = nn.Conv2d(32, 64, kernel_size=(3,3))
    self.l3 = nn.Linear(1600, 10)

  def __call__(self, x:Tensor) -> Tensor:
    x = self.l1(x).relu().max_pool2d((2,2))
    x = self.l2(x).relu().max_pool2d((2,2))
    return self.l3(x.flatten(1).dropout(0.5))

In [4]:
from tinygrad.nn.datasets import mnist
X_train, Y_train, X_test, Y_test = mnist()
print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)
# (60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar

(60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar


In [5]:
model = Model()
acc = (model(X_test).argmax(axis=1) == Y_test).mean()
# NOTE: tinygrad is lazy, and hasn't actually run anything by this point
print(acc.item())  # ~10% accuracy, as expected from a random model

0.08509999513626099


In [6]:
optim = nn.optim.Adam(nn.state.get_parameters(model))
batch_size = 128
def step():
  Tensor.training = True  # makes dropout work
  samples = Tensor.randint(batch_size, high=X_train.shape[0])
  X, Y = X_train[samples], Y_train[samples]
  optim.zero_grad()
  loss = model(X).sparse_categorical_crossentropy(Y).backward()
  optim.step()
  return loss

In [7]:
import timeit
timeit.repeat(step, repeat=5, number=1)
#[0.08268719699981375,
# 0.07478952900009972,
# 0.07714716600003158,
# 0.07785399599970333,
# 0.07605237000007037]

[12.849196879193187,
 4.837077775038779,
 3.8938264248427004,
 3.8316094540059566,
 3.9391237311065197]

In [8]:
from tinygrad import GlobalCounters, Context
GlobalCounters.reset()
with Context(DEBUG=2): step()

scheduled 45 kernels
*** CLANG      1 E_n11                                     arg  1 mem  0.06 GB tm      5.65us/     0.01ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__imul__']
*** CLANG      2 E_n12                                     arg  1 mem  0.06 GB tm      2.90us/     0.01ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__imul__']
*** CLANG      3 E_n6                                      arg  1 mem  0.06 GB tm      2.25us/     0.01ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['randint']
*** CLANG      4 r_20000_15000_3_4                         arg  1 mem  0.06 GB tm     24.36us/     0.04ms (     0.00 GFLOPS    9.9|9.9     GB/s) ['__getitem__']
*** CLANG      5 r_10_10n1                                 arg  1 mem  0.06 GB tm      2.29us/     0.04ms (     0.15 GFLOPS    0.0|0.0     GB/s) ['sparse_categorical_crossentropy']
*** CLANG      6 E_n10                                     arg  2 mem  0.06 GB tm      2.50us/     0.04ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__rsub__']
*** CL

In [9]:
from tinygrad import TinyJit
jit_step = TinyJit(step)

In [10]:
import timeit
timeit.repeat(jit_step, repeat=5, number=1)
# [0.2596786549997887,
#  0.08989566299987928,
#  0.0012115650001760514,
#  0.001010227999813651,
#  0.0012164899999334011]

[4.839767877943814,
 4.177499732002616,
 3.759829655988142,
 3.783605223055929,
 3.144151008920744]

In [11]:
for step in range(7000):
  loss = jit_step()
  if step%100 == 0:
    Tensor.training = False
    acc = (model(X_test).argmax(axis=1) == Y_test).mean().item()
    print(f"step {step:4d}, loss {loss.item():.2f}, acc {acc*100.:.2f}%")

step    0, loss 2.87, acc 73.23%
step  100, loss 0.28, acc 95.23%
step  200, loss 0.10, acc 96.73%
step  300, loss 0.09, acc 97.44%
step  400, loss 0.10, acc 97.58%
step  500, loss 0.16, acc 97.80%
step  600, loss 0.04, acc 97.96%
step  700, loss 0.05, acc 97.97%
step  800, loss 0.15, acc 97.93%
step  900, loss 0.16, acc 98.01%
step 1000, loss 0.05, acc 98.03%
step 1100, loss 0.15, acc 98.17%
step 1200, loss 0.18, acc 97.85%
step 1300, loss 0.21, acc 98.09%
step 1400, loss 0.13, acc 98.28%
step 1500, loss 0.09, acc 98.37%
step 1600, loss 0.07, acc 98.38%
step 1700, loss 0.13, acc 98.60%
step 1800, loss 0.15, acc 98.31%
step 1900, loss 0.10, acc 98.41%
step 2000, loss 0.11, acc 98.45%
step 2100, loss 0.18, acc 98.42%


KeyboardInterrupt: 